Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [51]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

print("Import successful")

Import successful


First reload the data we generated in `1_notmnist.ipynb`.

In [114]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  #train_dataset = train_dataset[1:200]
  #train_labels = train_labels[1:200]
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [115]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [54]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [92]:
#Very useful discussion here:
#https://discussions.udacity.com/t/deep-learning-assignment-2-2-fullyconnected-cannot-improve-the-test-accuracy-with-relu-layer-in-sgd/221268/6

batch_size = 128
num_hidden_nodes = 1024
l2_regularisation_penalty = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  hidden = tf.nn.relu(logits)
  out_layer = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=out_layer)
    + l2_regularisation_penalty * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out_layer)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [93]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 658.486023
Minibatch accuracy: 3.9%
Validation accuracy: 25.7%
Minibatch loss at step 500: 190.868317
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Minibatch loss at step 1000: 115.752991
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Minibatch loss at step 1500: 70.198982
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 2000: 42.572510
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 2500: 25.818384
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
Minibatch loss at step 3000: 15.658152
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Test accuracy: 78.5%


With l2_regularisation_penalty = 0 (no regularisation) -> accuracy = 89.3%

With l2_regularisation_penalty = 0.0005 -> accuracy = 91.1%

With l2_regularisation_penalty = 0.001 -> accuracy = 93.5%

With l2_regularisation_penalty = 0.005 -> accuracy = 92.0%

With l2_regularisation_penalty = 0.01 -> accuracy = 90.7%

With l2_regularisation_penalty = 0.05 -> accuracy = 85.5%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Before regularisation reducing training samples to just 200 reduces final test accuracy from 89.1% -> 78%
After regularisation (L2 = 0.001) reducing training samples to just 200 reduces final test accuracy from 93.5% -> 78.5%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [109]:
#Very useful discussion here:
#https://discussions.udacity.com/t/deep-learning-assignment-2-2-fullyconnected-cannot-improve-the-test-accuracy-with-relu-layer-in-sgd/221268/6

batch_size = 128
num_hidden_nodes = 1024
l2_regularisation_penalty = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  hidden = tf.nn.relu(logits)
  hidden_drop = tf.nn.dropout(hidden, keep_prob)
  
  out_layer = tf.matmul(hidden_drop, weights2) + biases2
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=out_layer)
    + l2_regularisation_penalty * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out_layer)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [110]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 874.533936
Minibatch accuracy: 4.7%
Validation accuracy: 14.4%
Minibatch loss at step 500: 319.747650
Minibatch accuracy: 95.3%
Validation accuracy: 73.7%
Minibatch loss at step 1000: 307.936646
Minibatch accuracy: 99.2%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 304.995483
Minibatch accuracy: 98.4%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 302.646667
Minibatch accuracy: 97.7%
Validation accuracy: 75.2%
Minibatch loss at step 2500: 300.502563
Minibatch accuracy: 96.1%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 296.083313
Minibatch accuracy: 99.2%
Validation accuracy: 75.5%
Test accuracy: 82.5%


Learning rate = 0.5 (default)//n
keep_prob = 1 -> 93.7%
keep_prob = 0.75 -> 93.0%
keep_prob = 0.5 -> 92.0%

Learning rate = 0.3
keep_prob = 1 -> 91.6%
keep_prob = 0.5 -> 90.4%

Learning rate = 0.01
keep_prob = 1 -> 87.5%
keep_prob = 0.5 -> 90.8%

Learning rate = 0.01, keep_prob = 0.5, num samples = 200 -> 82.5% -> seems to help better against overfitting

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [119]:
#Very useful discussion here:
#https://discussions.udacity.com/t/deep-learning-assignment-2-2-fullyconnected-cannot-improve-the-test-accuracy-with-relu-layer-in-sgd/221268/6

batch_size = 128
num_hidden_nodes = 1024 * 2
l2_regularisation_penalty = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  hidden = tf.nn.relu(logits)
  hidden_drop = tf.nn.dropout(hidden, keep_prob)
  
  out_layer = tf.matmul(hidden_drop, weights2) + biases2
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=out_layer)
    + l2_regularisation_penalty * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out_layer)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [120]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1251.593506
Minibatch accuracy: 10.9%
Validation accuracy: 32.9%
Minibatch loss at step 500: 615.805054
Minibatch accuracy: 78.1%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 545.589539
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 523.190674
Minibatch accuracy: 79.7%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 494.495667
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 482.357819
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 468.423767
Minibatch accuracy: 79.7%
Validation accuracy: 87.0%
Test accuracy: 92.6%


In [121]:
#learning_rate = tf.train.exponential_decay(0.5, global_step, 100, 0.95) -> 91.1%
#learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.95) -> 91.2%
#Increase hidden nodes 1024 -> 2048 --> 92.6% :)